### Openai

#### Load the API key and relevant Python libaries.

In [2]:
import openai
import os
import json 
import requests
from pprint import pprint
import json
import tiktoken
import openai
import requests
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored
GPT_MODEL = "gpt-3.5-turbo-0613"

In [3]:
from dotenv import load_dotenv, find_dotenv
from openai import OpenAI
_ = load_dotenv(find_dotenv())
openai.api_key  = os.getenv('OPENAI_API_KEY')

In [4]:
def count_token(input, model_name=GPT_MODEL, debug=False):
        encoding = tiktoken.encoding_for_model(model_name)
        codex = " ".join(input.splitlines())  # Join lines into a single string
        num_tokens = len(encoding.encode(codex))
        return num_tokens


### Completion Api

In [7]:
from openai import OpenAI
client = OpenAI()

response = client.completions.create(
  model="gpt-3.5-turbo-instruct",
  prompt="Write a tagline for an ice cream shop."
)

In [8]:
response.choices[0].text

'\n\n"Satisfy Your Sweet Tooth with Every Scoop!"'

### Chat Completions

In [9]:
client = OpenAI()

completion = client.chat.completions.create(
  model=GPT_MODEL,
  messages=[
    {"role": "system", "content": "You are an expert in writing taglines for ice cream shops. you are creative and original."},
    {"role": "user", "content": "Write a tagline for an ice cream shop."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='"Scoops of Joy, Happiness in Every Bite!"', role='assistant', function_call=None, tool_calls=None)


In [10]:
pprint(completion.choices[0].message.content)

'"Scoops of Joy, Happiness in Every Bite!"'


### Few shot Prompting 

In [11]:
from openai import OpenAI
client = OpenAI()

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {"role": "assistant", "content": "The Los Angeles Dodgers won the World Series in 2020."},
    {"role": "user", "content": "Where was it played?"}
  ]
)

In [12]:
response.choices[0].message.content

'The 2020 World Series was played in Arlington, Texas at Globe Life Field, the home stadium of the Texas Rangers.'

### delimiter and context

In [13]:
articles = """List of 2022 Winter Olympics medal winners

==Curling==

{{main|Curling at the 2022 Winter Olympics}}
{|{{MedalistTable|type=Event|columns=1|width=225|labelwidth=200}}
|-valign="top"
|Men<br/>{{DetailsLink|Curling at the 2022 Winter Olympics – Men's tournament}}
|{{flagIOC|SWE|2022 Winter}}<br/>[[Niklas Edin]]<br/>[[Oskar Eriksson]]<br/>[[Rasmus Wranå]]<br/>[[Christoffer Sundgren]]<br/>[[Daniel Magnusson (curler)|Daniel Magnusson]]
|{{flagIOC|GBR|2022 Winter}}<br/>[[Bruce Mouat]]<br/>[[Grant Hardie]]<br/>[[Bobby Lammie]]<br/>[[Hammy McMillan Jr.]]<br/>[[Ross Whyte]]
|{{flagIOC|CAN|2022 Winter}}<br/>[[Brad Gushue]]<br/>[[Mark Nichols (curler)|Mark Nichols]]<br/>[[Brett Gallant]]<br/>[[Geoff Walker (curler)|Geoff Walker]]<br/>[[Marc Kennedy]]
|-valign="top"
|Women<br/>{{DetailsLink|Curling at the 2022 Winter Olympics – Women's tournament}}
|{{flagIOC|GBR|2022 Winter}}<br/>[[Eve Muirhead]]<br/>[[Vicky Wright]]<br/>[[Jennifer Dodds]]<br/>[[Hailey Duff]]<br/>[[Mili Smith]]
|{{flagIOC|JPN|2022 Winter}}<br/>[[Satsuki Fujisawa]]<br/>[[Chinami Yoshida]]<br/>[[Yumi Suzuki]]<br/>[[Yurika Yoshida]]<br/>[[Kotomi Ishizaki]]
|{{flagIOC|SWE|2022 Winter}}<br/>[[Anna Hasselborg]]<br/>[[Sara McManus]]<br/>[[Agnes Knochenhauer]]<br/>[[Sofia Mabergs]]<br/>[[Johanna Heldin]]
|-valign="top"
|Mixed doubles<br/>{{DetailsLink|Curling at the 2022 Winter Olympics – Mixed doubles tournament}}
|{{flagIOC|ITA|2022 Winter}}<br/>[[Stefania Constantini]]<br/>[[Amos Mosaner]]
|{{flagIOC|NOR|2022 Winter}}<br/>[[Kristin Skaslien]]<br/>[[Magnus Nedregotten]]
|{{flagIOC|SWE|2022 Winter}}<br/>[[Almida de Val]]<br/>[[Oskar Eriksson]]
|}

Curling at the 2022 Winter Olympics

==Results summary==

===Women's tournament===

====Playoffs====

=====Gold medal game=====

''Sunday, 20 February, 9:05''
{{#lst:Curling at the 2022 Winter Olympics – Women's tournament|GM}}
{{Player percentages
| team1 = {{flagIOC|JPN|2022 Winter}}
| [[Yurika Yoshida]] | 97%
| [[Yumi Suzuki]] | 82%
| [[Chinami Yoshida]] | 64%
| [[Satsuki Fujisawa]] | 69%
| teampct1 = 78%
| team2 = {{flagIOC|GBR|2022 Winter}}
| [[Hailey Duff]] | 90%
| [[Jennifer Dodds]] | 89%
| [[Vicky Wright]] | 89%
| [[Eve Muirhead]] | 88%
| teampct2 = 89%
}}"""


In [14]:
system_message =f"""Use the provided articles delimited by xml tags to answer questions. If the answer cannot be found in the articles, write I could not find an answer.
articles: < {articles} > 
"""

In [15]:
count_token(system_message)

850

In [26]:
user_message ="""Question: Who won the gold medal in women's curling at the 2022 Winter Olympics?"""

In [27]:
count_token(user_message)

20

In [28]:
from openai import OpenAI
def get_gpt_response(system_message, user_message):
    client = OpenAI()
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": system_message
            },
            {
                "role": "user",
                "content": user_message
            }
        ],
        temperature=0,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].message.content


In [29]:
response = get_gpt_response(system_message, user_message)

In [30]:
response

"The gold medal in women's curling at the 2022 Winter Olympics was won by the team from Great Britain (GBR)."